In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## مرحلہ 1: ساختہ نتائج کے لیے Pydantic ماڈلز کی تعریف کریں

یہ ماڈلز وہ **اسکیمہ** متعین کرتے ہیں جو ایجنٹس واپس کریں گے۔ `response_format` کو Pydantic کے ساتھ استعمال کرنے سے درج ذیل فوائد حاصل ہوتے ہیں:
- ✅ محفوظ قسم کے ساتھ ڈیٹا نکالنا
- ✅ خودکار توثیق
- ✅ آزاد متن کے جوابات سے کوئی تجزیہ کی غلطیاں نہیں
- ✅ فیلڈز کی بنیاد پر آسان مشروط روٹنگ


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## مرحلہ 2: ہوٹل بکنگ ٹول بنائیں

یہ ٹول وہ ہے جسے **availability_agent** کمرے دستیاب ہونے کی جانچ کے لیے کال کرے گا۔ ہم `@ai_function` ڈیکوریٹر استعمال کرتے ہیں تاکہ:
- Python فنکشن کو AI کے ذریعے قابل استعمال ٹول میں تبدیل کریں
- LLM کے لیے خودکار JSON اسکیمہ تیار کریں
- پیرامیٹر کی تصدیق کو سنبھالیں
- ایجنٹس کے ذریعے خودکار کالنگ کو فعال کریں

اس ڈیمو کے لیے:
- **اسٹاک ہوم، سیئٹل، ٹوکیو، لندن، ایمسٹرڈیم** → کمرے دستیاب ہیں ✅
- **دیگر تمام شہر** → کمرے دستیاب نہیں ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## مرحلہ 3: روٹنگ کے لیے کنڈیشن فنکشنز کی وضاحت کریں

یہ فنکشنز ایجنٹ کے جواب کا جائزہ لیتے ہیں اور ورک فلو میں کس راستے پر جانا ہے، اس کا فیصلہ کرتے ہیں۔

**اہم طریقہ کار:**
1. چیک کریں کہ پیغام `AgentExecutorResponse` ہے
2. ساختی آؤٹ پٹ (Pydantic ماڈل) کو پارس کریں
3. روٹنگ کو کنٹرول کرنے کے لیے `True` یا `False` واپس کریں

ورک فلو ان شرائط کا جائزہ **کناروں** پر لے گا تاکہ فیصلہ کیا جا سکے کہ اگلا کون سا ایگزیکیوٹر استعمال کرنا ہے۔


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## مرحلہ 4: کسٹم ڈسپلے ایگزیکیوٹر بنائیں

ایگزیکیوٹرز وہ ورک فلو اجزاء ہیں جو تبدیلیاں یا ضمنی اثرات انجام دیتے ہیں۔ ہم `@executor` ڈیکوریٹر استعمال کرتے ہیں تاکہ ایک کسٹم ایگزیکیوٹر بنایا جا سکے جو حتمی نتیجہ دکھائے۔

**اہم تصورات:**
- `@executor(id="...")` - ایک فنکشن کو ورک فلو ایگزیکیوٹر کے طور پر رجسٹر کرتا ہے
- `WorkflowContext[Never, str]` - ان پٹ/آؤٹ پٹ کے لیے ٹائپ ہنٹس
- `ctx.yield_output(...)` - ورک فلو کا حتمی نتیجہ فراہم کرتا ہے


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## مرحلہ 5: ماحول کے متغیرات لوڈ کریں

LLM کلائنٹ کو ترتیب دیں۔ یہ مثال کام کرتی ہے:
- **GitHub ماڈلز** (GitHub ٹوکن کے ساتھ مفت درجے)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## مرحلہ 6: AI ایجنٹس کے ساتھ منظم نتائج بنائیں

ہم **تین خاص ایجنٹس** بناتے ہیں، ہر ایک کو `AgentExecutor` میں لپیٹا جاتا ہے:

1. **availability_agent** - ٹول کے ذریعے ہوٹل کی دستیابی چیک کرتا ہے
2. **alternative_agent** - متبادل شہروں کی تجویز دیتا ہے (جب کمرے دستیاب نہ ہوں)
3. **booking_agent** - بکنگ کی حوصلہ افزائی کرتا ہے (جب کمرے دستیاب ہوں)

**اہم خصوصیات:**
- `tools=[hotel_booking]` - ایجنٹ کو ٹول فراہم کرتا ہے
- `response_format=PydanticModel` - منظم JSON آؤٹ پٹ کو یقینی بناتا ہے
- `AgentExecutor(..., id="...")` - ورک فلو کے استعمال کے لیے ایجنٹ کو لپیٹتا ہے


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## مرحلہ 7: ورک فلو کو مشروط کناروں کے ساتھ بنائیں

اب ہم `WorkflowBuilder` کا استعمال کرتے ہوئے گراف کو مشروط راستہ کے ساتھ تعمیر کرتے ہیں:

**ورک فلو کی ساخت:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**اہم طریقے:**
- `.set_start_executor(...)` - انٹری پوائنٹ مقرر کرتا ہے
- `.add_edge(from, to, condition=...)` - مشروط کنارہ شامل کرتا ہے
- `.build()` - ورک فلو کو حتمی شکل دیتا ہے


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## مرحلہ 8: ٹیسٹ کیس 1 چلائیں - شہر بغیر دستیابی کے (پیرس)

آئیے **بغیر دستیابی** کے راستے کو جانچتے ہیں، پیرس میں ہوٹلوں کی درخواست کر کے (جو ہماری سیمولیشن میں کوئی کمرے نہیں رکھتا)۔


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## مرحلہ 9: ٹیسٹ کیس 2 چلائیں - شہر جس میں دستیابی موجود ہے (اسٹاک ہوم)

اب آئیے **دستیابی** کے راستے کو جانچتے ہیں، اسٹاک ہوم میں ہوٹلوں کی درخواست کرکے (جو ہمارے سیمولیشن میں کمرے رکھتا ہے)۔


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## اہم نکات اور اگلے اقدامات

### ✅ آپ نے کیا سیکھا:

1. **WorkflowBuilder پیٹرن**
   - انٹری پوائنٹ کو ڈیفائن کرنے کے لیے `.set_start_executor()` استعمال کریں
   - مشروط روٹنگ کے لیے `.add_edge(from, to, condition=...)` استعمال کریں
   - ورک فلو کو حتمی شکل دینے کے لیے `.build()` کال کریں

2. **مشروط روٹنگ**
   - کنڈیشن فنکشنز `AgentExecutorResponse` کا جائزہ لیتے ہیں
   - روٹنگ کے فیصلے کرنے کے لیے ساختہ آؤٹ پٹس کو پارس کریں
   - کسی ایج کو فعال کرنے کے لیے `True` واپس کریں، اسے چھوڑنے کے لیے `False`

3. **ٹول انٹیگریشن**
   - Python فنکشنز کو AI ٹولز میں تبدیل کرنے کے لیے `@ai_function` استعمال کریں
   - ایجنٹس ضرورت پڑنے پر خودکار طور پر ٹولز کو کال کرتے ہیں
   - ٹولز JSON واپس کرتے ہیں جسے ایجنٹس پارس کر سکتے ہیں

4. **ساختہ آؤٹ پٹس**
   - ٹائپ سیف ڈیٹا ایکسٹریکشن کے لیے Pydantic ماڈلز استعمال کریں
   - ایجنٹس بناتے وقت `response_format=MyModel` سیٹ کریں
   - جوابات کو `Model.model_validate_json()` کے ساتھ پارس کریں

5. **کسٹم ایکزیکیوٹرز**
   - ورک فلو کے اجزاء بنانے کے لیے `@executor(id="...")` استعمال کریں
   - ایکزیکیوٹرز ڈیٹا کو تبدیل کر سکتے ہیں یا سائیڈ ایفیکٹس انجام دے سکتے ہیں
   - ورک فلو کے نتائج پیدا کرنے کے لیے `ctx.yield_output()` استعمال کریں

### 🚀 حقیقی دنیا میں استعمال:

- **سفر کی بکنگ**: دستیابی چیک کریں، متبادل تجویز کریں، آپشنز کا موازنہ کریں
- **کسٹمر سروس**: مسئلے کی قسم، جذبات، یا ترجیح کی بنیاد پر روٹنگ کریں
- **ای کامرس**: انوینٹری چیک کریں، متبادل تجویز کریں، آرڈرز پروسیس کریں
- **مواد کی نگرانی**: زہریلے اسکورز یا یوزر فلیگز کی بنیاد پر روٹنگ کریں
- **منظوری کے ورک فلو**: رقم، یوزر رول، یا رسک لیول کی بنیاد پر روٹنگ کریں
- **کثیر مرحلہ پروسیسنگ**: ڈیٹا کے معیار یا مکمل ہونے کی بنیاد پر روٹنگ کریں

### 📚 اگلے اقدامات:

- مزید پیچیدہ شرائط شامل کریں (متعدد معیار)
- ورک فلو اسٹیٹ مینجمنٹ کے ساتھ لوپس نافذ کریں
- دوبارہ استعمال کے قابل اجزاء کے لیے سب ورک فلو شامل کریں
- حقیقی APIs (ہوٹل بکنگ، انوینٹری سسٹمز) کے ساتھ انٹیگریٹ کریں
- ایرر ہینڈلنگ اور فال بیک راستے شامل کریں
- بلٹ ان ویژولائزیشن ٹولز کے ساتھ ورک فلو کو ویژولائز کریں



---

**ڈس کلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا خامیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
